# Questions -duplicate data classification using sklearn and RNN

In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-question-pairs/train.csv.zip
/kaggle/input/quora-question-pairs/sample_submission.csv.zip
/kaggle/input/quora-question-pairs/test.csv
/kaggle/input/quora-question-pairs/test.csv.zip
/kaggle/input/quora-duplicate/quora_duplicate_questions_new.csv


In [23]:
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.metrics import mean_squared_error, confusion_matrix
from zipfile import ZipFile as unzip



In [24]:
train_path_zip = "/kaggle/input/quora-question-pairs/train.csv.zip"
sub_path = "/kaggle/input/quora-question-pairs/sample_submission.csv.zip"
test_path = "/kaggle/input/quora-question-pairs/test.csv"
test_path_zip = "/kaggle/input/quora-question-pairs/test.csv.zip"
validation_path = "/kaggle/working/csv_quora_data_valid.csv"

output_dir = "/kaggle/working/"

# local_path = "/kaggle/input/quora-duplicate/quora_duplicate_questions_new.csv"

In [ ]:
datasets = pd.read_csv(train_path_zip)

datasets.dropna(inplace=True)

train_dataset,valid_dataset  = datasets.iloc[:int(datasets.shape[0]*0.8),:],datasets.iloc[int(datasets.shape[0]*0.8):,:]

test_dataset = pd.read_csv(test_path_zip,low_memory=False)


valid_dataset.to_csv(validation_path)
train_path = "/kaggle/working/train.csv"
train_dataset.to_csv(train_path)
# datasets.head(1)


In [ ]:

print(f"Shape of train dataset:\t{ datasets.shape }\n")
print(f"Columns of datasets:\t{ datasets.columns}\n")
print(f"Validation dataset shape:\t{valid_dataset.shape}\n")
print(f"Training datasets\t{train_dataset.shape}\n")
print(f"First row Question1:\n{datasets.iloc[0]['question1']}\n")
print(f"First row Question2:\n{datasets.iloc[0]['question2']}\n")
print(f"Duplicate --0 or not --1:\n{datasets.iloc[0]['is_duplicate']}")

In [ ]:
tf_datasets = tf.data.experimental.CsvDataset(train_path, [ tf.string,tf.string,tf.int32],header=True, select_cols=[4,5,6])
valid_tf_datasets = tf.data.experimental.CsvDataset(validation_path, [tf.string,tf.string,tf.int32],header=True, select_cols=[4,5,6])
test_tf_datasets = tf.data.experimental.CsvDataset(test_path, [tf.string,tf.string],header=True, select_cols=[1,2])
valid_tf_datasets, test_tf_datasets, tf_datasets

In [ ]:
def print_data(data =tf_datasets):
    for i in data.take(1):
            print(i)
    # pd.read_csv("./csv_quora_data.csv")

In [ ]:
print("Total  3 datasets in pandas and 3 in tensor")

def get_all_pd_datasets():
    print(f'All Datasets are:\t train_dataset,valid_dataset, test_dataset')
    return train_dataset,valid_dataset, test_dataset

def get_all_tf_datasets():
    print(f'All Datasets are:\ttf_datasets, valid_tf_datasets, test_tf_datasets')
    return tf_datasets, valid_tf_datasets, test_tf_datasets


In [ ]:
SHUFFLE_SIZE = 10**5
BATCH_SIZE = 64
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
import regex as re
pat = re.compile("[.!@#$%^&*()><:;""''?/\*0-9]")

def tf_preprocess(input_data):
    input_data =input_data.map(lambda c,d,x: (tf.strings.lower(c),tf.strings.lower(d),x))
    data = input_data.map(lambda a,c,d: (tf.strings.regex_replace(a,"[.!@#$%^&*()><:;""''?/\*0-9]",""),tf.strings.regex_replace(c,"[.!@#$%^&*()><:;""''?/\*0-9]",""),d))
    data = data.cache().shuffle(SHUFFLE_SIZE).batch(BATCH_SIZE).prefetch(1)
    return data

def tf_test_preprocess(input_data):
    input_data =input_data.map(lambda c,d:( tf.strings.lower(c),tf.strings.lower(d)))
    data = input_data.map(lambda a,c: (tf.strings.regex_replace(a,"[.!@#$%^&*()><:;""''?/\*0-9]",""),tf.strings.regex_replace(c,"[.!@#$%^&*()><:;""''?/\*0-9]","")))
    data = data.batch(BATCH_SIZE).prefetch(1)    
    return data
tf_datasets= tf_datasets.apply(tf_preprocess)

valid_tf_datasets= valid_tf_datasets.apply(tf_preprocess)
test_tf_datasets = test_tf_datasets.apply(tf_test_preprocess)


In [ ]:
l = pd.read_csv(local_path)
l.columns
# tf_datasets
# print_data(tf_datasets)

In [ ]:
'''
To create vocab of most frequent words in datasets, here we first divide tf_datastes into datasets , each one for each 
column questions. 
then concatenate and using tensorflow text vectorization word dictionary

where every is key and mapped to it's frequency. Here, only considering most frequent 1000 words
'''

tf_text_dataset = tf_datasets.map(lambda qid1,qid2,y: ((qid1,qid2),y))
tf_text_dataset= tf_text_dataset.cache().shuffle(SHUFFLE_SIZE).prefetch(1)


tf_text_new = tf_datasets.map(lambda qid1, qid2,y: (qid1))
tf_text_new = tf_text_new.cache().prefetch(1)

tf_text_new2 = tf_datasets.map(lambda qid1, qid2,y: (qid2))
tf_text_new2 = tf_text_new2.cache().prefetch(1)


In [ ]:
text_df = tf_text_new.concatenate(tf_text_new2)
text_df = text_df.cache().prefetch(1)


In [ ]:
def print_data2(df=text_df):
    for i in df.take(1):
        if type(i)==tuple:
            print(i[0].shape, i[0][0])
        else:
            print(len(i), i.shape, i.numpy()[0])
print_data2()

In [ ]:
VOCAB_SIZE = 10000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(text_df.map(lambda text: text))

In [ ]:
vocab = np.array(encoder.get_vocabulary())
for example, label in tf_text_dataset.take(1):
    print("text:", example[0][0], example[1][0])
    print()
    print("label:",label[0])
encoded_example = encoder(example[0])[:3].numpy()
encoded_example.shape

In [ ]:
for n in range(3):
    print("Original: ", example[1][n].numpy())
    print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
    print()

# (encoded_example).shape

In [ ]:
print(f"most_freq_words_dictionary limit 10\t{vocab[:10]}") 

In [ ]:
model = keras.Sequential([
                          encoder,
                          keras.layers.Embedding(input_dim=len(encoder.get_vocabulary()),mask_zero=True,output_dim=64
                                                 ),
                          keras.layers.Bidirectional(keras.layers.LSTM(64)),
                          keras.layers.Dense(64,activation="relu"),
                          keras.layers.Dense(1)
])

print([layer.supports_masking for layer in model.layers])


sample_text = ('what is name president'
                'how do I become seo expert' )
predictions = model.predict(np.array([sample_text]))
print(predictions[0])


padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

In [ ]:

valid_tf_dataset = valid_tf_datasets.map(lambda qid1,qid2,y: ((qid1,qid2),y))
valid_tf_dataset= valid_tf_dataset.cache().shuffle(SHUFFLE_SIZE).prefetch(1)

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
save_model_path = "/kaggle/working/model"
saving_model = keras.callbacks.ModelCheckpoint(
    save_model_path, monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=True, mode='auto', save_freq='epoch',
    options=None
)
early = keras.callbacks.EarlyStopping(patience=10)
history = model.fit(tf_text_dataset, epochs=20,
                    validation_data=valid_tf_dataset,callbacks=[saving_model,early],
                    validation_steps=30)

In [ ]:
import matplotlib.pyplot as plt
model.load_weights(save_model_path)

print(f"Model accuray\t{model.evaluate(valid_tf_dataset)}")
print("It is clear that it is higly overfitting and need of heavy regularization is required by our model")
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

plot_graphs(history,'accuracy')

In [ ]:
for (text1,text2),label in valid_tf_dataset.take(1):
    print(len(text1), label.shape)
    print(text1[0], text2[0], label[0])

In [ ]:
# , label
m1 = (model.predict((text1,text2)) > 0.5).astype("int32").reshape(1,-1)[0]; m2 = label

print(m1,m2)
print(f"mean squred error of predictions\t{mean_squared_error(m2,m1)}")

In [ ]:
# predictions or evalution on test dataset
def get_predictions_print(count=9, data=test_tf_datasets, label=None, print_stats=True):
    print(f"Predictions on only single bacth of size {BATCH_SIZE}:\t{data}\n")
    some_examples = next(data.as_numpy_iterator())
    if label:
        y_pred = (model.predict((some_examples[0],some_examples[1])) > 0.5).astype("int32").reshape(1,-1)[0]
    else:
        y_pred = (model.predict(some_examples) > 0.5).astype("int32").reshape(1,-1)[0]
    if print_stats:
        for i in range(count):
            print(i+1, some_examples[0][i],'\n')
            print(i+1, some_examples[1][i],'\n')
            print(i+1,"Predicted label:", y_pred[i],'\n')
            if label:
                print(f"True label:\t{some_examples[2][i]}\n")
    if label:
        m = mean_squared_error(some_examples[2], y_pred)
        print(f"Mean squared error:\t{m}")
        return y_pred,m
    return y_pred

get_predictions_print()

In [ ]:
get_predictions_print(data=valid_tf_datasets, label=True)


test_predictions = max(model.predict_proba(test_tf_dataset), axis=1)

1.0